In [ ]:
# https://www.lottomatica.it/lotterie/lotto
# https://www.igt.it/STORICO_ESTRAZIONI_LOTTO/storico.zip
# https://living-sun.com/it/python/705402-how-can-i-extract-a-closed-zip-with-python-python-zipfile.html


In [1]:
# importazione delle librerie
import requests
import zipfile
import csv
import pandas as pd
import numpy as np
import funzioni as fn
from pprint import pprint
from itertools import combinations, permutations


CLASSI

In [2]:
class Numero:

    def __init__(self, n, nome_file_ruota):

        self.precedente_ritardo = 0                     # l'ultimo ritardo conosciuto
        self.frequenze = []
        self.frequenza_attuale = 0
        self.numero = n                                 # deve essere un vettore di massimo 5 numeri
        self.dataset = fn.leggi_da_csv(nome_file_ruota) # è il dataframe di ricerca informazioni             
        self.ruota = self.dataset[1][1]
        self.ritardi = []
        self.scompensi = []
        self.scomp = 0
        self.uscite_totali = 0
        

    def info(self):
        # impostazione del vettore che memorizza il numero della estrazione in cui esce, e serve per tutti i calcoli successivi
        temp = []
        #print(self.dataset[0][2:])
        tipo = len(self.numero)
        if tipo == 1:
            for i in range(len(self.dataset)):
                #print(self.dataset[i][2:])
                if str(self.numero[0]) in self.dataset[i][2:]:
                    temp.append(i+1)
        elif tipo > 1:
            # definisce l'ambo, il terno, la quaterna o la cinquina da cercare
            temp_n = []
            for i in self.numero:
                temp_n.append(str(i))
            temp_n = tuple(temp_n)
            #print(temp_n)

            for i in range(len(self.dataset)):
                comb = list(permutations(self.dataset[i][2:], tipo))
                #print(comb)
                
                if temp_n in comb:
                    temp.append(i+1)
            #print(temp)
            
        if len(temp) == 0:
            self.precedente_ritardo = len(self.dataset) + 1
            self.ultimo_ritardo = len(self.dataset) + 1
            self.ritardo_max = len(self.dataset) + 1
            self.frequenza_attuale = 0
            self.previsione = np.NaN
            self.scomp = np.NaN
            
        elif len(temp) == 1: 
            self.precedente_ritardo = 0
            self.ultimo_ritardo = temp[0]
            self.uscite_totali = len(temp)

            # calcolo delle scompensazioni
            for i in range(1, len(temp)):
                r = temp[i] - temp[i-1]                 # Ritardo del numero nel momento in cui è uscito
                self.ritardi.append(r)                  # archivio dei ritardi
                n_estr = temp[i]                        # numero della estrazione del numero nel momento in cui è uscito
                freq = int(n_estr / len(temp[:i]))      # frequenza nel momento in cui è uscito
                self.frequenze.append(freq)
                self.scomp += (r - freq)                # scompensazione nel momento in cui è uscito
                self.scompensi.append(self.scomp)

            self.ritardo_max = temp[0]
            self.frequenza_attuale = int((len(self.dataset)+1)/(len(temp)))
            self.previsione = self.frequenza_attuale - self.ultimo_ritardo

        else:
            self.precedente_ritardo = temp[len(temp)-1] - temp[len(temp)-2] # Rutaro precedente all'ultima uscita
            self.ultimo_ritardo = len(self.dataset) - temp[len(temp)-1]     # ritardo all'ultima uscita
            self.uscite_totali = len(temp)
            
            # calcolo delle scompensazioni
            for i in range(1, len(temp)):
                r = temp[i] - temp[i-1]                 # Ritardo del numero nel momento in cui è uscito
                self.ritardi.append(r)                  # archivio dei ritardi
                n_estr = temp[i]                        # numero della estrazione del numero nel momento in cui è uscito
                freq = int(n_estr / len(temp[:i]))      # frequenza nel momento in cui è uscito
                self.frequenze.append(freq)
                self.scomp += (r - freq)                # scompensazione nel momento in cui è uscito
                self.scompensi.append(self.scomp)

            self.ritardo_max = max(self.ritardi)                        # massimo ritardo fatto dal numero
            self.frequenza_attuale = int((len(self.dataset)+1)/(len(temp)))   # frequenza attuale
            self.previsione = self.frequenza_attuale - self.ultimo_ritardo      # previsione di uscita calcolata sulla frequenza e il ritardo attiale

    def mostra_info(self):

        #valori = {
        #    "Numero" :             self.numero,
        #    "Ruota" :              self.ruota,
        #    "N. uscite" :           self.uscite_totali
        #    "Ritardo Precedente" : self.precedente_ritardo,
        #    "Ritardo Attuale" :    self.ultimo_ritardo,
        #    "Ritardo Massimo" :    self.ritardo_max,
        #    "Frequenza" :          self.frequenza_attuale,
        #    "Previsione" :         self.previsione,
        #    "Scompensazione":      self.scomp
        #}

        valori = [
            self.numero,
            self.ruota,
            self.uscite_totali,
            self.precedente_ritardo,
            self.ultimo_ritardo,
            self.ritardo_max,
            self.frequenza_attuale,
            self.previsione,
            self.scomp
            ]
    
        return valori
    
    def print_info(self):

        print(
            "\n","Numero", self.numero,
            "\n","Ruota", self.ruota,
            "\n","N. uscite", self.uscite_totali,
            "\n","Ritardo Precedente", self.precedente_ritardo,
            "\n","Ritardo Attuale", self.ultimo_ritardo,
            "\n","Ritardo Massimo", self.ritardo_max,
            "\n","Frequenza", self.frequenza_attuale,
            "\n","Previsione", self.previsione,
            "\n","Scompensazione", self.scomp
        )

    def spia(self, numero_estrezioni):
        tipo = len(self.numero)
        if tipo == 1:
            a = []
            for i in range(len(self.dataset)- numero_estrezioni):
                if str(self.numero[0]) in self.dataset[i][2:]:
                    for j in range(0, numero_estrezioni):
                        a += self.dataset[i+j][2:]
        
        risultato = trova_duplicati(a)

        return risultato





FUNZIONI

In [3]:
# Funzione per creare il dataset dei numeri in formato oggetto e richiamabili
def dataset_numeri(data):
    numeri = []
    for n in range(1, 91):
        t = Numero([n], data)
        numeri.append([n, t])
    
    return numeri

In [4]:
# Funzione per creare una tabella con tutte le statistiche di ogni numero
def print_numeri(data):
    numeri = []
    for n in range(1, 91):
        t = Numero([n], data)
        t.info()
        numeri.append(t.mostra_info())
    
    return numeri


In [5]:
# Funzione per creare una tabella con tutte le statistiche di ogni numero come un database pandas
def print_dataframe_from_numeri(data):
    numeri = print_numeri(data)
    df = pd.DataFrame(numeri, columns=[
                                        "Numero", 
                                        "Ruota", 
                                        "N. Uscite",
                                        "Ritardo Precedente",
                                        "Ritardo Attuale",
                                        "Ritardo Massimo", 
                                        "Frequenza", 
                                        "Previsione",
                                        "Scompensazione"
                                        ]
                    ).set_index(
                                [
                                    "Numero", 
                                    "Ruota"
                                ]
                            )

    return df

In [9]:
def trova_duplicati(lista):

    nya_lista = set(lista)
    counting = []
    for element in nya_lista:
        count = 0
        for i in lista:
            if element == i:
                count += 1
        counting.append([count, element])

    return counting

CODICE

In [ ]:
numeri = []
for i in range(1, 91):
    numeri.append(i)

temp = Numero([23], "BA.csv")
temp.info()
#print(temp.uscite_totali)
temp.print_info()
temp.mostra_info()
spie = sorted(temp.spia(5))
#print(spie[-20:])
spie_ordinate = []
for i in range(1, 21):
    spie_ordinate.append(spie[-i][1])
print('20 Numeri spia a 5 estrazioni: \n',spie_ordinate)




In [82]:
# Funzione per trovare i numeri spia in ordine 
def numeri_spia_ordinati(numero, ruota, n_est_spia, n_spie):

    temp = Numero([numero], ruota+".csv")
    temp.info()
    spie = sorted(temp.spia(n_est_spia))
    spie_ordinate = []
    for i in range(1, (n_spie+1)):
        spie_ordinate.append(int(spie[-i][1]))

    return spie_ordinate

In [83]:
# Funzione per calcolare tutti i numeri spia per ogni estrazioni
def numeri_spia_ultima_estrazione(ruota, n_est_spia, n_spie):
    ultima_riga = fn.leggi_da_csv(ruota+'.csv')[-1]
    elenco = []
    for i in ultima_riga[2:]:
        temp = numeri_spia_ordinati(int(i), ruota, n_est_spia, n_spie)
        for i in temp:
            elenco.append(i)
    #print(elenco)
    elenco_ordinato = sorted(trova_duplicati(elenco))[-n_spie:]
    spie_totali = []
    for i in elenco_ordinato:
        spie_totali.append(i[1])

    return spie_totali


In [38]:
# Funzione per trovare i 20 numeri migliori come frequenza
def migliori_numeri(ruota, n_migliori):

    lista = []
    for i in range(1, 91):
        temp = Numero([i], ruota+".csv")
        temp.info()
        lista.append([temp.uscite_totali, i])

    new_lista = sorted(lista)
    numeri = []
    for i in range(len(new_lista[-n_migliori:])):
        numeri.append(new_lista[i][1])

    return numeri

In [48]:
# Funzione per trovare i migliori 3 ambi su un tot di numeri
def migliori_ambi(numeri, ruota):

    ambi = list(combinations(numeri, 2))

    migliori = []
    for i in ambi:

        temp = Numero(i, ruota+".csv")
        temp.info()
        if temp.ultimo_ritardo < 14:
            migliori.append([temp.uscite_totali, temp.ultimo_ritardo, i])

    temp = sorted(migliori)[-3:]
    ambi_da_abbinare = []
    for i in range(len(temp)):
        ambi_da_abbinare.append(temp[i][2])

    return ambi_da_abbinare

In [86]:
# Programma su tutte le ruote

ruote = ["BA", "FI", "MI", "NA", "PA", "RM", "TO", "VE"]
for i in ruote:

    print('Calcolo sulla ruota di', i)
    n1 = numeri_spia_ultima_estrazione(i, 5, 20)
    n2 = migliori_numeri(i, 20)
    print('Ambi da numeri spia', migliori_ambi(n1, i))
    print('Ambi da migliori numeri',migliori_ambi(n2, i))
    print('\n')

Calcolo sulla ruota di BA
Ambi da numeri spia [(85, 59), (15, 59), (26, 89)]
Ambi da migliori numeri [(25, 6), (43, 80), (38, 72)]


Calcolo sulla ruota di FI
Ambi da numeri spia [(30, 77), (79, 76), (57, 30)]
Ambi da migliori numeri [(45, 34), (37, 12)]


Calcolo sulla ruota di MI
Ambi da numeri spia [(30, 84), (30, 51), (14, 15)]
Ambi da migliori numeri [(22, 9), (21, 72), (25, 45)]


Calcolo sulla ruota di NA
Ambi da numeri spia [(87, 52), (54, 76), (43, 83)]
Ambi da migliori numeri [(10, 24)]


Calcolo sulla ruota di PA
Ambi da numeri spia [(61, 81), (2, 86), (56, 87)]
Ambi da migliori numeri [(46, 69), (10, 50), (10, 62)]


Calcolo sulla ruota di RM
Ambi da numeri spia [(18, 62), (45, 73), (5, 82)]
Ambi da migliori numeri [(53, 2), (37, 28), (44, 37)]


Calcolo sulla ruota di TO
Ambi da numeri spia [(88, 37), (12, 60), (61, 88)]
Ambi da migliori numeri [(25, 63), (57, 14)]


Calcolo sulla ruota di VE
Ambi da numeri spia [(70, 68), (72, 68), (77, 74)]
Ambi da migliori numeri [(62, 

In [33]:
import pandas_ta as ta
import matplotlib.pyplot as plt

df = pd.DataFrame()

temp = Numero([23], "BA.csv")
#print(temp.scompensi)
temp.info()
temp.print_info()
#print(temp.ritardi)
n = str("close")
if temp.uscite_totali > 0:
    df[n] = temp.scompensi
    sma10 = df.ta.sma(length=10)
    sma30 = df.ta.sma(length=30)
    df = pd.concat([df[n], sma10, sma30], axis=1)
    #print(list(df))
    df['scomp_sigLine'] = np.where(df['SMA_10'] > df['SMA_30'], 1, 0 )
    df["Status"] = np.where(df['scomp_sigLine'] == 0, 1, 0)

    qf = pd.DataFrame()
    qf["Ritardi"] = temp.ritardi


    pf = pd.concat([df, qf], axis=1)


    pf["Frequenze"] = temp.frequenze
    pf.to_csv("stats.csv")
else:
    print(f"{temp.numero} non è mai uscito")

#print(list(pf))


 Numero [23] 
 Ruota BA 
 N. uscite 375 
 Ritardo Precedente 9 
 Ritardo Attuale 12 
 Ritardo Massimo 114 
 Frequenza 17 
 Previsione 5 
 Scompensazione -235


In [85]:
def plot_graph(pf):

    pf=pf.astype(float)
    plt.rcParams["figure.figsize"] = (36, 30)
    #plt.rcParams["savefig.format"] = 'png'  

    # plotting di tutte le curve 
    plt.plot(pf['close'][:], color='blue', label='ritardo')
    plt.plot(pf["SMA_10"][:], color='red', label='SMA_10')
    plt.plot(pf["SMA_30"][:], color='green', label='SMA_30')

    # configuraione degli assi
    plt.xlabel('Numero uscita',fontsize=18)
    plt.ylabel('Riardo',fontsize=18)

    # plot signals EMA_RSI_ATR
    #plt.plot(df.index, df['BuyEMA_RSI_ATR'], '^', markersize = 15, color = 'purple', label='BuyEMA_RSI_ATR') # plot ‘Buy’ signals
    #plt.plot(df.index, df['SellEMA_RSI_ATR'], 'v', markersize = 15, color = 'black', label='SellEMA_RSI_ATR') # plot ‘Sell’ signals

    # configurazioni del titolo, leggende, griglia
    plt.title('GRAFICO', fontsize = 20)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_graph(df)

In [ ]:
# prendere i migliori 20 estratti per ruota in base al numero di volte che sono usciti (i piu' prequenti)
# nelle ultime estrazioni del mese precedente l'estrazione verifico che ambi sono usciti nei 20 numeri
# i migliori 3 ambi le rigioco in quartine per altre 13 estrazioni successive

In [32]:
lista = []
for i in range(1, 91):
    temp = Numero([i], "BA.csv")
    temp.info()
    lista.append([temp.uscite_totali, i])
    #print(temp.uscite_totali)
    #temp.print_info()
    #temp.mostra_info()

new_lista = sorted(lista)
numeri = []
for i in range(len(new_lista[-20:])):
    #print(new_lista[i][1])
    numeri.append(new_lista[i][1])

print(numeri)

ambi = list(combinations(numeri, 2))

#print(ambi)
migliori = []
for i in ambi:

    temp = Numero(i, "BA.csv")
    #print(temp.scompensi)
    temp.info()
    #temp.print_info()
    if temp.ultimo_ritardo < 14:
        print(i, temp.ultimo_ritardo, temp.uscite_totali)
        migliori.append([temp.uscite_totali, temp.ultimo_ritardo, i])

temp = sorted(migliori)[-3:]
ambi_da_abbinare = []
for i in range(len(temp)):
    ambi_da_abbinare.append(temp[i][2])

quartine = list(combinations(ambi_da_abbinare, 2))
print(quartine)


[24, 45, 25, 21, 7, 41, 6, 38, 57, 43, 64, 72, 39, 42, 28, 32, 52, 67, 80, 63]
(25, 6) 1 17
(25, 64) 1 14
(6, 64) 1 14
(38, 72) 6 20
(57, 43) 4 14
(57, 80) 4 16
(43, 80) 4 18
[((25, 6), (43, 80)), ((25, 6), (38, 72)), ((43, 80), (38, 72))]
